In [1]:
# !pip install numpy 
# !pip install --upgrade tensorflow
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.model_selection import train_test_split
import fastai
# import tensorflow as tf
import sys
print(sys.executable, sys.version)

/Users/robertkasumba/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/Users/robertkasumba/opt/anaconda3/bin/python 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]


In [2]:
svhn = fetch_openml("SVHN_small")
n_samples = 1000

In [3]:
X = svhn["data"][0:n_samples].values.reshape(-1, 32, 32, 3) / 255.0
y = svhn["target"][0:n_samples]
y = y.astype(np.int32)
svhn = None


In [4]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val= train_test_split(X,y)


In [5]:
X_train.shape

(750, 32, 32, 3)

In [6]:

X_train_labeled,X_train_unlabeled, y_labeled,_ = train_test_split(X_train,y_train,test_size = 0.5)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import random

class MixMatch(nn.Module):
    def __init__(self, model, loss_fn, num_classes, beta, augmentations):
        super().__init__()
        self.model = model
        self.loss_fn = loss_fn
        self.num_classes = num_classes
        self.beta = beta
        self.augmentations = augmentations
        
    def mixup(self, x, y, alpha=1.0):
        """
        Generates mixed examples and corresponding labels.
        """
        # If alpha is positive, generate the mixing parameter from a beta distribution
        if alpha > 0:
            lam = np.random.beta(alpha, alpha)
        else:
            lam = 1

        # Get the batch size
        batch_size = x.size()[0]
        
        # Generate a random permutation of the indices of the batch
        index = torch.randperm(batch_size).cuda()

        # Compute the mixed examples
        mixed_x = lam * x + (1 - lam) * x[index,:]
        
        # Compute the mixed labels
        mixed_y = lam * y + (1 - lam) * y[index]
        
        # Return the mixed examples and labels
        return mixed_x, mixed_y

    def augment(self, x):
        """
        Applies data augmentation to the input.
        """
        if self.augmentations is not None:
            x = self.augmentations(x)
        return x

    def sharpening(self, predictions, temperature=0.5):
        """
        Sharpens the predictions using temperature scaling.
        """
        return predictions ** (1 / temperature)
    
    def augment(self, x):
        """
        Applies data augmentation to the input.
        """
        # If augmentations are specified, apply them to the input
        if self.augmentations is not None:
            x = self.augmentations(x)
            
        # Return the augmented input
        return x

    def sharpening(self, predictions, temperature=0.5):
        """
        Sharpens the predictions using temperature scaling.
        """
        # Compute the sharpened predictions
        return predictions ** (1 / temperature)
    
    def forward(self, x_l, y_l, x_u):
        """
        Performs the MixMatch forward pass.
        """
        # Augment the labeled examples and labels
        x_l, y_l = self.augment(x_l), y_l
        
        # Augment the unlabeled examples
        x_u = self.augment(x_u)
        
        # Generate mixed examples and corresponding labels
        mixed_x, mixed_y = self.mixup(x_l, y_l, self.beta)
        
        # Predict the labels for mixed examples
        mixed_preds = self.model(mixed_x)
        
        # Sharpen the predictions for mixed examples
        mixed_preds = self.sharpening(mixed_preds, temperature=0.5)
        
        # Predict the labels for unlabeled examples
        unlabeled_preds = self.model(x_u)
        
        # Sharpen the predictions for unlabeled examples
        unlabeled_preds = self.sharpening(unlabeled_preds, temperature=0.5)
         # Compute the loss for labeled examples
        labeled_loss = self.loss_fn(mixed_preds, mixed_y)
        
        # Compute the loss for unlabeled examples
        unlabeled_loss = -torch.mean(torch.log_softmax(unlabeled_preds, dim=1), dim=0)[y_l]
        unlabeled_loss = torch.mean(unlabeled_loss)
        
        # Return the sum of labeled and unlabeled losses
        return labeled_loss + self.beta * unlabeled_loss
    
import torch.nn as nn
import torch.nn.functional as F

# Define the model architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # first hidden layer, 128 neurons
        self.fc2 = nn.Linear(128, 64)  # second hidden layer, 64 neurons
        self.fc3 = nn.Linear(64, 32)  # third hidden layer, 32 neurons
        self.fc4 = nn.Linear(32, 10)  # output layer, 10 neurons

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # flatten the input to 1D
        x = F.relu(self.fc1(x))  # first hidden layer with ReLU activation
        x = F.relu(self.fc2(x))  # second hidden layer with ReLU activation
        x = F.relu(self.fc3(x))  # third hidden layer with ReLU activation
        x = self.fc4(x)  # output layer
        return F.log_softmax(x, dim=1)  # log_softmax activation for multi-class classification

# Initialize the model
model = Net()


In [ ]:
# Convert the arrays to PyTorch tensors
X_train_labeled = torch.tensor(np.array(X_train_labeled))
y_labeled = torch.tensor(np.array(y_labeled))
X_train_unlabeled = torch.tensor(np.array(X_train_unlabeled))




In [ ]:
# Create a dataset from the labeled samples
labeled_dataset = TensorDataset(X_train_labeled, y_labeled)

# Create a data loader from the labeled dataset
labeled_data_loader = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True)

# Create a dataset from the unlabeled samples
unlabeled_dataset = TensorDataset(X_train_unlabeled)

# Create a data loader from the unlabeled dataset
unlabeled_data_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
import torch
import torch.optim as optim
from torchvision import transforms
from mixmatch import MixMatch

# Set the optimizer to Adam with learning rate 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the number of classes in the target variable to 10
num_classes = 10

# Set the beta parameter to 0.5
beta = 0.5

# Set the loss function to CrossEntropyLoss
loss_fn = torch.nn.CrossEntropyLoss()

# Define the augmentations to be applied to the input data
augmentations = transforms.Compose([
    transforms.RandomCrop(32, padding=4), # Crop the input image to 32x32 and pad if necessary
    transforms.RandomHorizontalFlip(), # Randomly flip the image horizontally
    transforms.RandomRotation(15), # Randomly rotate the image by up to 15 degrees
    transforms.ToTensor(), # Convert the image to a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize the image
])

# Initialize the MixMatch loss function
mixmatch_loss = MixMatch(model, loss_fn, num_classes, beta, augmentations)

# Train the model for 100 epochs
for epoch in range(100):
    for i, (x, y) in enumerate(train_loader): # Use a data loader to process the training data in batches
        # Zero the gradients
        optimizer.zero_grad()

        # Compute the loss
        loss = mixmatch_loss(model, x1, y1, x2, y2, lam)
        
        # Compute the gradients and update the parameters
        loss.backward()
        optimizer.step()
        
        # Print the loss every 100 batches
        if i % 100 == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')
